In [1]:
import json
import pickle

import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# Load intents JSON file:
with open("intents.json") as json_file:
    data = json.load(json_file)
    

In [3]:
# A place to store our training data:
training_sentences = []
training_labels = []
labels = []
responses = []


# Loop through each intent:
for intent in data['intents']:
    # Loop through each pattern:
    for pattern in intent['patterns']:
        # Store the pattern:
        training_sentences.append(pattern)
        # Store the associated tag:
        training_labels.append(intent['tag'])

    # Store possible responses:
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])


num_classes = len(labels)

In [4]:
# Implement label encoding:
label_encoder = LabelEncoder()
label_encoder.fit(training_labels)
training_labels = label_encoder.transform(training_labels)


In [5]:
# Vectorize the training data using Tokenization:
vocabulary_size = 1000
embedding_dimension = 16
max_length = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocabulary_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences,
                                 maxlen=max_length,
                                 truncating='post')

In [6]:
# Define Neural Network architecture:
model = Sequential()
model.add(Embedding(vocabulary_size,
                    embedding_dimension,
                    input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 15)                495       
Total params: 18,095
Trainable params: 18,095
Non-trainable params: 0
_________________________________________________________________


In [8]:
epochs = 400
history = model.fit(padded_sequences,
                    np.array(training_labels),
                    epochs=epochs,
                    workers=4)

Epoch 1/500
4/4 [==============================] - 0s 997us/step - loss: 2.7062 - accuracy: 0.1157
Epoch 2/500
4/4 [==============================] - 0s 748us/step - loss: 2.7027 - accuracy: 0.1405
Epoch 3/500
4/4 [==============================] - 0s 997us/step - loss: 2.6990 - accuracy: 0.1653
Epoch 4/500
4/4 [==============================] - 0s 997us/step - loss: 2.6961 - accuracy: 0.1157
Epoch 5/500
4/4 [==============================] - 0s 997us/step - loss: 2.6923 - accuracy: 0.1074
Epoch 6/500
4/4 [==============================] - 0s 2ms/step - loss: 2.6892 - accuracy: 0.1074
Epoch 7/500
4/4 [==============================] - 0s 1ms/step - loss: 2.6849 - accuracy: 0.1074
Epoch 8/500
4/4 [==============================] - 0s 1ms/step - loss: 2.6808 - accuracy: 0.1074
Epoch 9/500
4/4 [==============================] - 0s 997us/step - loss: 2.6766 - accuracy: 0.1074
Epoch 10/500
4/4 [==============================] - 0s 1ms/step - loss: 2.6721 - accuracy: 0.1074
Epoch 11/500
4/4 

In [9]:
# Save the neural network model:
model.save("chat_model")

# Save the fitted tokenizer:
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle,
                protocol=pickle.HIGHEST_PROTOCOL)
    

# Save the fitted label encoder:
with open("label_encoder.pickle", "wb") as enc_file:
    pickle.dump(label_encoder, enc_file,
                protocol=pickle.HIGHEST_PROTOCOL)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: chat_model\assets
